In [1]:
import os

import pandas as pd
import tiktoken

from readers import read_indexer_entities, read_indexer_reports
# from graphrag.query.llm.oai.chat_openai import ChatOpenAI
# from graphrag.query.llm.oai.typing import OpenaiApiType
# from graphrag.query.structured_search.global_search.community_context import (
#     GlobalCommunityContext,
# )
# from graphrag.query.structured_search.global_search.search import GlobalSearch

In [2]:
token_encoder = tiktoken.get_encoding("cl100k_base")

In [3]:
# parquet files generated from indexing pipeline
INPUT_DIR = "./index_results/atelectasis/"
COMMUNITY_REPORT_TABLE = "create_final_community_reports"
ENTITY_TABLE = "create_final_nodes"
ENTITY_EMBEDDING_TABLE = "create_final_entities"

# community level in the Leiden community hierarchy from which we will load the community reports
# higher value means we use reports from more fine-grained communities (at the cost of higher computation cost)
COMMUNITY_LEVEL = 2

In [4]:
df = pd.read_parquet("./index_results/atelectasis/create_base_text_units.parquet")

In [5]:
entity_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_TABLE}.parquet")
report_df = pd.read_parquet(f"{INPUT_DIR}/{COMMUNITY_REPORT_TABLE}.parquet")
entity_embedding_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_EMBEDDING_TABLE}.parquet")

reports = read_indexer_reports(report_df, entity_df, COMMUNITY_LEVEL)
entities = read_indexer_entities(entity_df, entity_embedding_df, COMMUNITY_LEVEL)
print(f"Report records: {len(report_df)}")
report_df.head()

Report records: 63


,community,full_content,level,rank,title,rank_explanation,summary,findings,full_content_json,id
0,59,# Lung Cancer Community\n\nThe Lung Cancer com...,2,7.5,Lung Cancer Community,The impact severity rating is high due to the ...,The Lung Cancer community consists of key enti...,[{'explanation': 'Lung Cancer serves as the ce...,"{\n ""title"": ""Lung Cancer Community"",\n ...",cd3260dd-8762-4079-ac0d-7c1422b5aa1c
1,60,# Regional Lymph Node in Lung Cancer\n\nThe co...,2,7.5,Regional Lymph Node in Lung Cancer,The impact severity rating is high due to the ...,The community is centered around the Regional ...,[{'explanation': 'The Regional Lymph Node clas...,"{\n ""title"": ""Regional Lymph Node in Lung C...",5412eeb1-9f43-4874-bbf3-ecd333fe1116
2,61,# Silicosis and Lung Diseases\n\nThe community...,2,7.5,Silicosis and Lung Diseases,The impact severity rating is high due to the ...,The community consists of entities related to ...,[{'explanation': 'Silicosis is a severe form o...,"{\n ""title"": ""Silicosis and Lung Diseases"",...",28323d38-c7fe-45e1-bee1-0fe059b545a4
3,62,# Lung Cancer and Squamous Cell Carcinoma\n\nT...,2,7.5,Lung Cancer and Squamous Cell Carcinoma,The impact severity rating is high due to the ...,The community consists of entities related to ...,[{'explanation': 'Squamous Cell Carcinoma is a...,"{\n ""title"": ""Lung Cancer and Squamous Cell...",2bb603b8-749c-4752-a7a5-4f948f6f5f73
4,63,# Operation: Dulce Mission and Cosmic Interact...,2,7.5,Operation: Dulce Mission and Cosmic Interactions,The impact severity rating is high due to the ...,The community is centered around the Operation...,[{'explanation': 'Operation: Dulce is the cent...,"{\n ""title"": ""Operation: Dulce Mission and ...",3c3781dc-e64c-4f86-aaeb-015a5f4a183f


In [6]:
from global_search.community_context import GlobalCommunityContext

In [7]:
context_builder = GlobalCommunityContext(
    community_reports=reports,
    entities=entities,  # default to None if you don't want to use community weights for ranking
    token_encoder=token_encoder,
)

In [8]:
context_builder_params = {
    "use_community_summary": False,  # False means using full community reports. True means using community short summaries.
    "shuffle_data": True,
    "include_community_rank": True,
    "min_community_rank": 0,
    "community_rank_name": "rank",
    "include_community_weight": True,
    "community_weight_name": "occurrence weight",
    "normalize_community_weight": True,
    "max_tokens": 12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
    "context_name": "Reports",
}

map_llm_params = {
    "max_tokens": 1000,
    "temperature": 0.0,
    "response_format": {"type": "json_object"},
}

reduce_llm_params = {
    "max_tokens": 2000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 1000-1500)
    "temperature": 0.0,
}

In [ ]:
import langchain

In [9]:
from verbs.template import (
         MAP_SYSTEM_PROMPT,
         # REDUCE_SYSTEM_PROMPT,
         GENERAL_KNOWLEDGE_INSTRUCTION,
         NO_DATA_ANSWER,
      )

In [10]:
from verbs.send_to import send_to_open_ai

In [11]:
from global_search.search import GlobalSearch
search_engine = GlobalSearch(
    send_to=send_to_open_ai,
    context_builder=context_builder,
    token_encoder=token_encoder,
    max_data_tokens=12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
    map_llm_params=map_llm_params,
    reduce_llm_params=reduce_llm_params,
    allow_general_knowledge=False,  # set this to True will add instruction to encourage the LLM to incorporate general knowledge in the response, which may increase hallucinations, but could be useful in some use cases.
    json_mode=True,  # set this to False if your LLM model does not support JSON mode.
    context_builder_params=context_builder_params,
    concurrent_coroutines=32,
    response_type="multiple paragraphs",  # free form text describing the response type and format, can be anything, e.g. prioritized list, single paragraph, multiple paragraphs, multiple-page report
)

In [15]:
result = search_engine.search(
    "What cause atelectasis?"
)

print(result.response)

Map response: %s {
    "points": [
        {
            "description": "Atelectasis can be caused by airway obstruction, lack of adequate inspiration, and other factors such as thorax compression in Flat-Chested Kitten Syndrome (FCKS) [Data: Reports (51, 17, 49)]",
            "score": 100
        }
    ]
}
Map response: %s {
    "points": [
        {
            "description": "Passive Atelectasis is a form of lung atelectasis caused by conditions such as pneumothorax and pleural effusion. Other contributing factors to the development of Passive Atelectasis include Diaphragmatic Abnormalities and Hypoventilation. Understanding these causes is crucial in evaluating the overall impact of atelectasis-related conditions [Data: Reports (37)]",
            "score": 100
        }
    ]
}
Map response: %s {
    "points": [
        {
            "description": "Atelectasis can be associated with conditions like Bronchiectasis, Tracheal Stenosis, and Pulmonary Atelectasis, indicating potential

In [16]:
# inspect the data used to build the context for the LLM responses
result.context_data["reports"]

,id,title,occurrence weight,content,rank
0,47,Respiratory Conditions and Symptoms,1.000000,# Respiratory Conditions and Symptoms\n\nThe c...,8.0
1,20,Atelectasis and Related Entities Community,1.000000,# Atelectasis and Related Entities Community\n...,7.5
2,39,Pulmonary Embolism and Thromboembolism,0.821429,# Pulmonary Embolism and Thromboembolism\n\nTh...,7.5
3,19,Round Atelectasis and Related Entities,0.392857,# Round Atelectasis and Related Entities\n\nTh...,7.5
4,10,Lung Nodules and Imaging Technology,0.321429,# Lung Nodules and Imaging Technology\n\nThe c...,7.5
5,45,Cystic Fibrosis and Related Terms,0.285714,# Cystic Fibrosis and Related Terms\n\nThe com...,8.5
6,31,Respiratory System Entities Community,0.285714,# Respiratory System Entities Community\n\nThe...,8.5
7,42,Lung Cancer Community,0.285714,# Lung Cancer Community\n\nThe Lung Cancer com...,7.5
8,27,Consolidation and Atelectasis Community,0.250000,# Consolidation and Atelectasis Community\n\nT...,7.5
9,52,Lung Collapse and Associated Conditions,0.250000,# Lung Collapse and Associated Conditions\n\nT...,7.5


In [17]:
# inspect number of LLM calls and tokens
print(f"LLM calls: {result.llm_calls}. LLM tokens: {result.prompt_tokens}")

LLM calls: 4. LLM tokens: 30166
